In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

import datetime
from datetime import timedelta
# !pip install holidays
import holidays

os.chdir("/content/drive/MyDrive/3. Grad School/LG Aimers")

In [ ]:
# 데이터 불러오기
data = pd.read_csv("DATA/train/train.csv")

#### 파생변수 생성

In [ ]:
# 월(month) -> 계절 매핑 딕셔너리
month_to_season = {
    1: "Winter", 2: "Winter", 12: "Winter",
    3: "Spring", 4: "Spring", 5: "Spring",
    6: "Summer", 7: "Summer", 8: "Summer",
    9: "Autumn", 10: "Autumn", 11: "Autumn"}

season_weights = {
    "Winter" : 11.4,
    "Spring": 6.5,
    "Summer" : 6.3,
    "Autumn" : 18.4}

# 월별 가중치 매핑
monthly_weights = {
    1: 2.2, 2: 1.8, 3: 0.3,
    4: 1.01, 5: 0.7, 6: 0.8,
    7: 0.5, 8: 0.5, 9: 0.8,
    10: 1.55, 11: 1.03, 12: 1.4}

# 요일별 가중치 매핑
weekly_weights = {
    "Monday": 0.78, "Tuesday": 0.85, "Wednesday": 0.81,
    "Thursday": 9.9, "Friday": 1.2, "Saturday": 1.53,
    "Sunday": 1.3}

In [ ]:
class Make_Variables():
        def __init__(self, data = None, date = None, predict = 7, month_to_season = None, monthly_weights = None, weekly_weights = None):
            self.data = data
            self.date = date
            self.predict = predict
            self.month_to_season = month_to_season
            self.monthly_weights = monthly_weights
            self.weekly_weights = weekly_weights

        def update_kor_holidays(self):
            """국경일 추가"""
            kor_holidays = holidays.KR(years = [2023, 2024, 2025])
            kor_holidays.update({
                # datetime.date(2023,2,14) : "Valentine's Day",
                # datetime.date(2023,3,14) : "White Day",
                # datetime.date(2023,11,11) : "Pepero Day",
                # datetime.date(2024,2,14) : "Valentine's Day",
                # datetime.date(2024,3,14) : "White Day",
                # datetime.date(2024,11,11) : "Pepero Day",
                # datetime.date(2025,2,14) : "Valentine's Day",
                # datetime.date(2025,3,14) : "White Day",
                # datetime.date(2025,11,11) : "Pepero Day",

                datetime.date(2024,10,1) : "Temporary Holiday", # 국군의 날 임시공휴일
                datetime.date(2025,1,27) : "Temporary Holiday", # 설날 임시공휴일
                datetime.date(2025,3,3) : "Temporary Holiday", # 삼일절 대체공휴일
                datetime.date(2025, 5, 29) : "Election Period",
                datetime.date(2025, 5, 30) : "Election Period",
                datetime.date(2025, 6, 3) : "Presidential Election Day"})
            return kor_holidays

        def check_holidays(self, date, kor_holidays) -> int:
            """날짜 받아서 공휴일/주말 여부 출력"""
            # date = pd.Timestamp(date)
            if isinstance(date, pd.Series):
                check_holiday = date.dt.date.isin(kor_holidays)
                check_weekend = date.dt.weekday >= 5
            else:
                check_holiday = date.date() in kor_holidays
                check_weekend = date.weekday() >= 5
            is_holiday = (check_holiday | check_weekend)
            return is_holiday

        def get_sandwich_score(self, data, is_holiday_col) -> pd.DataFrame:
            """데이터프레임 기준으로 샌드위치 점수 계산"""
            data = data.reset_index(drop = True)
            data['is_sandwich'] = 0
            is_holiday = data[is_holiday_col].astype(int)
            for idx in range(len(data)):
                if idx == 0 or idx == len(data) - 1: # 첫날, 마지막 날
                    continue

                # 앞/뒤 하루씩 봤을 때 모두 휴일 -> 5점
                if (is_holiday.iloc[idx - 1] == 1) and (is_holiday.iloc[idx + 1] == 1): # 하루 전이랑 다음 날이 공휴일이면
                    data.iloc[idx, data.columns.get_loc('is_sandwich')] = 5

                # 앞/뒤 이틀씩 봤을 때 휴일 3일 -> 3점, 2일 -> 2점
                elif idx > 1 and idx < len(is_holiday) - 2: # 셋째날, 마지막에서 세 번째 날
                    start_idx = idx - 2
                    end_idx = idx + 2
                    nearby_holidays = (is_holiday.iloc[start_idx : end_idx + 1].sum() - is_holiday.iloc[idx])
                    if nearby_holidays == 3:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 3
                    elif nearby_holidays == 2:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 2
                    else:
                        data.iloc[idx, data.columns.get_loc('is_sandwich')] = 0
            return data

        def get_sandwich_score_for_dates(self, date, kor_holidays) -> int:
            """특정 날짜를 받아와서 앞뒤 날짜를 구하고, 샌드위치 점수 계산"""
            # 하루씩
            prev_date, next_date = date - timedelta(days = 1), date + timedelta(days = 1)
            prev_hol, next_hol = self.check_holidays(prev_date, kor_holidays), self.check_holidays(next_date, kor_holidays) # T/F Bool
            if prev_hol and next_hol: # 바로 다음 날들이 휴일이라면
                return 3
            days_offsets = [-2, -1, 1, 2] # 앞뒤로 이틀 살펴보기
            nearby_holidays = sum(self.check_holidays(date + timedelta(days = d), kor_holidays) for d in days_offsets)
            if nearby_holidays == 3: # 앞뒤 4일 중에 3일이 휴일이면
                return 2
            elif nearby_holidays == 2: # 앞뒤 4일 중에 2일이 휴일이면
                return 1
            else:
                return 0

        def get_season_weights(self, data = None, season_weights = season_weights):
            """계절별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['season_weight'] = data['season'].map(season_weights)
                return data

        def get_month_weights(self, data = None, monthly_weights = monthly_weights):
            """월별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['month_weight'] = data['month'].map(monthly_weights)
                return data

        def get_week_weights(self, data = None, weekly_weights = weekly_weights):
            """요일별 가중치 부여"""
            # 데이터프레임 들어오면
            if data is not None:
                data['week_weight'] = data['weekday'].map(weekly_weights)
                return data

        def get_prev_days(self, data, test_df = None, date = None, menu = None, howmany = 7):
            """
            일요일 날짜 받아와서 직전 주차의 일-토 매출수량 평균 계산
            주의 - test data에서 생성할 때는 참고할 데이터와 붙여넣을 데이터가 다름
            data : 참고할 데이터
            test_df : 참고할 데이터
            """
            if test_df is None:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = data[(data['영업일자'] >= prev_start) & (data['영업일자'] <= prev_end) & (data['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan

            # test data라면
            else:
                # 혹시 모르니까 검증
                if date.weekday() == 6:
                    # 이전 날짜들
                    prev_start = date - timedelta(days = howmany)
                    prev_end = date - timedelta(days = 1)
                    prev_data = test_df[(test_df['영업일자'] >= prev_start) & (test_df['영업일자'] <= prev_end) & (test_df['영업장명_메뉴명'] == menu)]
                    prev_avg = prev_data['매출수량'].mean()
                    prev_sd = prev_data['매출수량'].std()
                    # 첫 주 0으로 처리
                    if pd.isna(prev_avg):
                        prev_avg = 0
                    if pd.isna(prev_sd):
                        prev_sd = 0
                    week_end = date + timedelta(days = 6)
                    curr_mask = (data['영업일자'] >= date) & (data['영업일자'] <= week_end) & (data['영업장명_메뉴명'] == menu)
                    colname_mean = f"prev_avg_{howmany}"
                    colname_sd = f"prev_sd_{howmany}"
                    data.loc[curr_mask, colname_mean] = prev_avg
                    data.loc[curr_mask, colname_sd] = prev_sd
                    return data
                else:
                    return np.nan


        # train, test 공통
        def make_fund_variables(self, data, month_to_season = month_to_season):
            # 영업일자 -> datetime
            data['영업일자'] = pd.to_datetime(data['영업일자'])

            # 연, 월, 일, 요일 분리
            data['year'] = data['영업일자'].dt.year
            data['month'] = data['영업일자'].dt.month
            data['day'] = data['영업일자'].dt.day
            data['weekday'] = data['영업일자'].dt.day_name()
            data['weekday_enc'] = data['영업일자'].dt.weekday

            # 계절 변수 생성
            data['season'] = data['month'].map(month_to_season)

            # 연도 차이 변수 생성
            data['year_enc'] = data['year'] - 2023

            # 월, 일, 요일 사이클릭 변환
            data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
            data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)

            data['day_sin'] = np.sin(2 * np.pi * data['day'] / 31)
            data['day_cos'] = np.cos(2 * np.pi * data['day'] / 31)

            data['weekday_sin'] = np.sin(2 * np.pi * data['weekday_enc'] / 7)
            data['weekday_cos'] = np.cos(2 * np.pi * data['weekday_enc'] / 7)

            # 공휴일 확인
            kor_holidays = self.update_kor_holidays()
            check_holiday = data['영업일자'].dt.date.isin(kor_holidays)
            check_weekend = data['weekday'].isin(['Saturday', 'Sunday'])
            data['is_holiday'] = (check_holiday | check_weekend).astype(int) # 공휴일 + 주말
            data['holiday_name'] = data['영업일자'].dt.date.map(kor_holidays)

            return data

        # train의 입력 데이터
        def make_variables_train(self, data):
            data = self.make_fund_variables(data)
            kor_holidays = self.update_kor_holidays()

            ### 샌드위치 데이
            data = self.get_sandwich_score(data, 'is_holiday')

            # 샌드위치 - 첫날
            first = data['영업일자'].min()
            data.loc[data['영업일자'] == first, 'is_sandwich'] = self.get_sandwich_score_for_dates(first, kor_holidays)
            second = data['영업일자'].min() + timedelta(days = 1)
            data.loc[data['영업일자'] == second, 'is_sandwich'] = self.get_sandwich_score_for_dates(second, kor_holidays)

            # 샌드위치 - 마지막 날
            last = data['영업일자'].max()
            data.loc[data['영업일자'] == last, 'is_sandwich'] = self.get_sandwich_score_for_dates(last, kor_holidays)
            before = data['영업일자'].max() - timedelta(days = 1)
            data.loc[data['영업일자'] == before, 'is_sandwich'] = self.get_sandwich_score_for_dates(before, kor_holidays)

            # 샌드위치 포함한 공휴일
            data['is_holiday_sandwich'] = data['is_holiday'].astype(int) | (data['is_sandwich'] > 0).astype(int)

            ### 계절별 가중치
            data = self.get_season_weights(data, season_weights)

            ### 월별 가중치
            data = self.get_month_weights(data, monthly_weights)

            ### 요일별 가중치
            data = self.get_week_weights(data, weekly_weights)

            ### 직전 주차 평균
            sundays = data[data['weekday'] == "Sunday"][["영업일자", "영업장명_메뉴명"]].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 7)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 14)
                data = self.get_prev_days(data = data, date = date, menu = menu, howmany = 21)

            ### 영업장명, 메뉴명 분리
            if '영업장명_메뉴명' in data.columns:
                data[['영업장명', '메뉴명']] = data['영업장명_메뉴명'].str.split('_', expand = True)

            ### 음수 처리
            negative = data[data['매출수량'] < 0]

            for idx, row in negative.iterrows():
                num = row['매출수량']
                if num < -10:
                    date = row['영업일자']
                    menu = row['영업장명_메뉴명']
                    prev_date = pd.to_datetime(date) - pd.Timedelta(days = 1)
                    prev_row = data[(data['영업일자'] == prev_date) & (data['영업장명_메뉴명'] == menu)]

                    if prev_row.iloc[0]["매출수량"] >= abs(num):
                        data.loc[prev_row.index[0], '매출수량'] += num

            # 남은 건 전부 0으로
            data.loc[data['매출수량'] < 0, '매출수량'] = 0

            return data

        # 예측하고자 하는 날들
        def make_variables_test(self, date, test_df, predict):
            """
            date : 최종 날짜 (입력 7일 중 가장 마지막) - TimeStamp
            test_df : 예측할 때 참고해올 데이터 -> 이거로 직전 주차 평균 생성
            """
            date = pd.to_datetime(date)
            future_dates = [date + timedelta(days = i + 1) for i in range(predict)]
            future_df = pd.DataFrame({'영업일자' : future_dates})

            menus_df = (test_df[['영업장명_메뉴명']].drop_duplicates().reset_index(drop = True))
            future_df = future_df.merge(menus_df, how='cross')

            kor_holidays = self.update_kor_holidays()

            # 기본적인 변수들
            future_df = self.make_fund_variables(future_df)

            future_df['영업일자'] = pd.to_datetime(future_df['영업일자']).dt.normalize()

            # 샌드위치
            future_df['is_sandwich'] = future_df['영업일자'].apply(lambda d: self.get_sandwich_score_for_dates(d, kor_holidays))

             # 샌드위치 포함한 공휴일
            future_df['is_holiday_sandwich'] = future_df['is_holiday'].astype(int) | (future_df['is_sandwich'] > 0).astype(int)

            # 월별 가중치
            future_df = self.get_month_weights(future_df, monthly_weights)

            # 요일별 가중치
            future_df = self.get_week_weights(future_df, weekly_weights)

            # 직전 주차 평균 -> 이거는 test 까지 받아오고 생각해야 함..
            sundays =  future_df.loc[future_df['weekday'] == "Sunday", ['영업일자', '영업장명_메뉴명']].copy()
            for _, row in sundays.iterrows():
                date = row['영업일자']
                menu = row['영업장명_메뉴명']
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 7)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 14)
                future_df = self.get_prev_days(data = future_df, test_df = test_df, date = date, menu = menu, howmany = 21)

            return future_df

In [ ]:
# 그냥 전부 만들면 돼
mv = Make_Variables()
data = mv.make_variables_train(data)

In [ ]:
print("기존 공휴일 : ", data['is_holiday'].sum().item())
print("샌드위치 : ", data['is_sandwich'].sum().item())
print("샌드위치 포함 공휴일 : ", data['is_holiday_sandwich'].sum().item())

기존 공휴일 :  33389
샌드위치 :  579
샌드위치 포함 공휴일 :  33968


In [ ]:
import pickle
data.to_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_new.pickle")

#### 저장된 데이터 불러오기

In [ ]:
# 데이터 불러오기
data = pd.read_pickle("/content/drive/MyDrive/3. Grad School/LG Aimers/DATA/train_data_new.pickle")

In [ ]:
cols =  ["year_enc", "month_sin", "month_cos", "day_sin", "day_cos", "weekday_sin", "weekday_cos", "is_holiday", "is_sandwich", "is_holiday_sandwich",
         "season_weight", "month_weight", "week_weight", "prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]
enc_cols = ["season", "is_holiday", "is_holiday_sandwich"]
# lstm에서는 minmaxscaling 해주는 게 안전
num_cols = ["is_sandwich", "month_weight", "week_weight", "prev_avg_7", "prev_avg_14", "prev_avg_21", "prev_sd_7", "prev_sd_14", "prev_sd_21"]

#### 매출 여부 (분류)

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.metrics import f1_score
from typing import Dict, List, Optional, Any
from collections import defaultdict
import pickle
import joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.1 MB/s eta 0:00:00


In [ ]:
class ClassificationModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols

    def fit_model_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str]) -> pd.DataFrame:
        """
        data - 전체 dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        oof_parts = []

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 나중에 oof 저장할 때 사용할 것들
            org_idx = group_df.index
            oof_proba = np.full(len(group_df), np.nan)

            # x, y 분리
            x = group_df[cols]
            y = group_df["매출_여부"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 3)

            # 각 split 별로
            for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                # 수량 전부 0이거나 0 아닌 날 없으면 학습 불가
                if y_train.nunique() < 2:
                    print(f"{menu} 학습 불가")
                    continue

                # 범주형 변수 인코딩
                target_encoder = TargetEncoder()
                target_encoder.fit(x_train[enc_cols], y_train)

                x_train = pd.concat([
                    target_encoder.transform(x_train[enc_cols]),
                    x_train[[c for c in cols if c not in enc_cols]]
                ], axis = 1)

                x_val = pd.concat([
                    target_encoder.transform(x_val[enc_cols]),
                    x_val[[c for c in cols if c not in enc_cols]]
                ], axis = 1)

                # 모델 설정
                xgb_model = XGBClassifier(random_state = 1471)

                # 모델 학습
                xgb_model.fit(x_train, y_train)
                oof_proba[val_idx] = xgb_model.predict_proba(x_val)[:, 1]

            part = pd.DataFrame({
                "index": org_idx.values,
                "영업장명_메뉴명": menu,
                "y_true": group_df["매출_여부"].values,
                "y_proba": oof_proba
            }).set_index("index")

            oof_parts.append(part)

        oof_df = pd.concat(oof_parts).sort_index()
        return oof_df


    def tuning_cv(self, data, cols, enc_cols, param_grid):
        """
        메뉴별로 하이퍼파라미터 튜닝 진행 (성능 보고.. 추가 진행? )
        """
        best_params_by_menu = {}

        return None


    def tune_threshold(self, oof_df : pd.DataFrame, metric = 'f1') -> Dict[str, float]:
        """
        메뉴별로 OOF 데이터프레임을 받아와서
        threshold를 최적화
        metric으로는 f1 사용
        """
        best_thresholds = {}

        for menu, group_df in oof_df.groupby("영업장명_메뉴명"):
            y_true = group_df["y_true"].values
            y_proba = group_df["y_proba"].values

            # 초기값 진행
            best_score, best_thr = -1, 0.5
            for thr in np.linspace(0.05, 0.95, 51):
                y_pred = (y_proba >= thr).astype(int)
                score = f1_score(y_true, y_pred, zero_division = 0)
                if score > best_score:
                    best_score = score
                    best_thr = thr

            best_thresholds[menu] = best_thr

        return best_thresholds


    def get_final_model(self, data, cols, enc_cols, thresholds, hyperparameters_dict = None) -> Dict[str, Dict[str, Any]]:
        """
        메뉴별로 튜닝된 하이퍼파라미터, threshold를 반영하여
        최종 모델 적합
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values('영업일자')

            # 범주형 변수 처리 - 전체 데이터로
            target_encoder_full = TargetEncoder()
            group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출_여부'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출_여부"]

            # 하이퍼파라미터
            hyperparameters = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            # 모델 설정
            xgb_model_full = XGBClassifier(random_state = 1471, **hyperparameters)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                "encoder" : target_encoder_full,
                "threshold" : thresholds.get(menu, 0.5)}

        return models

    def fit_whole_model(self, data, cols, enc_cols) -> Dict[str, Dict[str, Any]]:
        oof_df = self.fit_model_cv(data, cols, enc_cols)
        print("CV 완료!")
        thresholds = self.tune_threshold(oof_df)
        print("threshold 탐색 완료!")
        models = self.get_final_model(data, cols, enc_cols, thresholds)
        return models

    def save_cls_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
classification = ClassificationModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 3/cls_models.pkl'

data_zero = data.copy()
data_zero['매출_여부'] = data_zero['매출수량'].apply(lambda x:1 if x > 0 else 0)

models = classification.fit_whole_model(data = data_zero, cols = cols, enc_cols = enc_cols)
classification.save_cls_model(models, model_path)

In [ ]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/cls_models.pkl'
classification = ClassificationModel()
models_class = classification.load_saved_model(model_path)

#### 매출 예측 (회귀)

In [ ]:
from xgboost import XGBRegressor
# ! pip install category_encoders
from category_encoders import TargetEncoder
from sklearn.model_selection import TimeSeriesSplit
from collections import defaultdict
from itertools import product
import pickle
import joblib

In [ ]:
class RegressionModel():
    def __init__(self, data = None, cols = None, enc_cols = None, model_path = None):
        self.data = data
        self.cols = cols
        self.enc_cols = enc_cols

    def smape_score(self, y_true, y_pred, eps = 1e-8):
        y_true = np.asarray(y_true, dtype=float)
        y_pred = np.asarray(y_pred, dtype=float)
        denom = np.abs(y_true) + np.abs(y_pred)
        denom = np.where(denom < eps, eps, denom)
        return 200.0 * np.mean(np.abs(y_true - y_pred) / denom)

    def tuning_cv(self, data : pd.DataFrame, cols : List[str], enc_cols : List[str], param_grid = None) -> Dict[str, Dict]:
        """
        메뉴별로 하이퍼파라미터 튜닝 진행
        """
        best_params_by_menu = {}

        if param_grid is None:
            param_grid = {
                "min_child_weight" : [1, 5],
                "max_depth" : [5, 6, 7],
                "subsample" : [0.7, 0.9],
                "n_estimators" : [100, 150],
                "learning_rate" : [0.03, 0.05]}

        keys = list(param_grid.keys())
        combos = list(product(*[param_grid[k] for k in keys]))

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            x = group_df[cols]
            y = group_df["매출수량"]

            # time series split
            tscv = TimeSeriesSplit(n_splits = 3)

            # 초기값
            best_smape = np.inf
            best_params = None

            for values in combos:
                params = dict(zip(keys, values))

                base_params = {"random_state" : 1478}
                base_params.update(params)

                fold_smapes : List[float] = []

                for fold, (train_idx, val_idx) in enumerate(tscv.split(x)):

                    x_train, y_train = x.iloc[train_idx], y.iloc[train_idx]
                    x_val, y_val = x.iloc[val_idx], y.iloc[val_idx]

                    # 데이터 수 적으면 학습 불가
                    if len(x_train) < 20:
                        print(f"{menu} 튜닝 불가)")
                        continue

                    # 범주형 변수 인코딩
                    target_encoder = TargetEncoder()
                    target_encoder.fit(x_train[enc_cols], y_train)

                    x_train = pd.concat([
                        target_encoder.transform(x_train[enc_cols]),
                        x_train[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                    x_val = pd.concat([
                        target_encoder.transform(x_val[enc_cols]),
                        x_val[[c for c in cols if c not in enc_cols]]
                    ], axis = 1)

                    # 모델 설정
                    xgb_model = XGBRegressor(**base_params)

                    # 모델 학습
                    xgb_model.fit(x_train, y_train)

                    pred = xgb_model.predict(x_val)
                    smape = self.smape_score(y_val, pred)
                    fold_smapes.append(smape)

                mean_smape = float(np.mean(fold_smapes))

                if mean_smape < best_smape:
                    best_smape = mean_smape
                    best_params = base_params

            if best_params is not None:
                best_params_by_menu[menu] = best_params

        return best_params_by_menu


    def get_final_model(self, data, cols, enc_cols, hyperparameters_dict : dict[str, dict[str, Any]] = None)  -> Dict[str, Dict[str, Any]]:
        """
        data, validation_reg - train, validation dataset
        cols - 전체 변수들
        enc_cols - 인코딩 진행할 변수들 (범주형)
        """
        models = {}

        for menu, group_df in data.groupby("영업장명_메뉴명"):

            group_df = group_df.sort_values("영업일자")

            # 데이터 수 적으면 학습 불가
            if len(group_df) < 10:
                print(f"{menu} 학습 불가")
                continue

            # 범주형 변수 처리
            target_encoder_full = TargetEncoder()
            group_df[enc_cols] = target_encoder_full.fit_transform(group_df[enc_cols], group_df['매출수량'])

            # x, y 분리
            x_full = group_df[cols]
            y_full = group_df["매출수량"]

            # 하이퍼파라미터 불러오기
            params = hyperparameters_dict.get(menu, {}) if hyperparameters_dict else {}

            base_params = {"random_state" : 1478}
            base_params.update(params)

            # 모델 설정
            xgb_model_full = XGBRegressor(**base_params)

            # 모델 학습
            xgb_model_full.fit(x_full, y_full)

            models[menu] = {
                "model" : xgb_model_full,
                "hyperparameters" : base_params,
                "encoder" : target_encoder_full
            }

        return models

    def fit_whole_model(self, data, cols, enc_cols) -> Dict[str, Dict[str, Any]]:
        best_params_by_menu = self.tuning_cv(data, cols, enc_cols)
        print("튜닝 완료!")
        models = self.get_final_model(data, cols, enc_cols, best_params_by_menu)
        return models

    def save_reg_model(self, models, model_path):
        joblib.dump(models, model_path)
        print("모델 저장 완료!")

    def load_saved_model(self, model_path):
        models = joblib.load(model_path)
        return models

In [ ]:
regression = RegressionModel()
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/reg_models.pkl'

data_notzero = data[data['매출수량'] > 0]

models = regression.fit_whole_model(data = data_notzero, cols = cols, enc_cols = enc_cols)
regression.save_reg_model(models, model_path)

튜닝 완료!
모델 저장 완료!


In [ ]:
# 다시 불러오기
model_path = '/content/drive/MyDrive/3. Grad School/LG Aimers/Models/Trial 6/reg_models.pkl'
regression = RegressionModel()
models_reg = regression.load_saved_model(model_path)

#### 예측하기

In [ ]:
class PredictionFunctions():
    def __init__(self, test_df = None, trained_models = None, test_prefix = None, cols = None, enc_cols = None, lookback = 28, predict = 7):
        self.test_df = test_df
        self.trained_models = trained_models
        self.test_prefix = test_prefix
        self.cols = cols
        self.enc_cols = enc_cols
        self.lookback = lookback
        self.predict = predict

    def predict_class(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출여부] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu]["encoder"]
            threshold = trained_models[store_menu].get("threshold", 0.5)

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            encoded = encoder.transform(future_df[enc_cols])
            encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
            future_df[enc_cols] = encoded_df
            x = future_df[cols]

            if hasattr(model, "classes_"):
                pos_idx = int(np.where(model.classes_ == 1)[0][0])
            else:
                pos_idx = 1

            proba = model.predict_proba(x)[:, pos_idx]
            y_hat = (proba >= threshold).astype(int)


            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, y_hat):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출여부': val
                })

        return pd.DataFrame(results)

    def predict_reg(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, lookback = 28, predict = 7):
        """
        Input : test_df - test data, trained_models - list(menu : {model, encoder}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """
        results = []

        for store_menu_tup, store_test in test_df.groupby(['영업장명_메뉴명']):
            store_menu = store_menu_tup[0]
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델 불러오기
            model = trained_models[store_menu]["model"]
            encoder = trained_models[store_menu]["encoder"]

            # 변수 추가하기
            mv = Make_Variables()
            store_test['영업일자'] = pd.to_datetime(store_test['영업일자'])
            store_test_sorted = store_test.sort_values('영업일자')
            last_date = store_test_sorted['영업일자'].iloc[-1]

            future_df = mv.make_variables_test(date = last_date, test_df = store_test, predict = 7)
            encoded = encoder.transform(future_df[enc_cols])
            encoded_df = pd.DataFrame(encoded, columns = enc_cols, index = future_df.index)
            future_df[enc_cols] = encoded_df
            future_df = future_df[cols]

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, model.predict(future_df)):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량': val
                })

        return pd.DataFrame(results)

    def predict_lstm(self, test_df, trained_models, test_prefix : str, cols : list, enc_cols : list, num_cols : list, lookback = 28, predict = 7, device = None):
        """
        Input : test_df - test data, trained_models - list(menu : { model}), cols - x 변수들
        Output : [영업일자, 영업장명_메뉴명, 매출수량] DataFrame
        """

        if device is None:
            device = getattr(self, "device", "cpu")
        device = torch.device(device)

        results = []

        # 매장, 메뉴별로 그룹화해서 예측
        for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
            # 훈련된 모델에 메뉴가 있는 경우만 진행
            if store_menu not in trained_models:
                continue

            # 모델, scaler 불러오기
            state = trained_models[store_menu]['model']
            meta = trained_models[store_menu]['meta']
            encoders = trained_models[store_menu]['encoders']
            features_scaler = trained_models[store_menu]['features_scaler']
            target_scaler = trained_models[store_menu]['target_scaler']

            # LSTM 모델 생성
            model = MultiOutputLSTM(
                input_dim=meta['input_dim'],
                hidden_dim=meta['hidden_dim'],
                num_layers=meta['num_layers'],
                output_dim=meta['output_dim']
            )

            model.load_state_dict(state)
            model = model.to(device).eval()

            # LSTM 입력으로 활용할 최근 lookback 만큼의 데이터 가져오기
            mv = Make_Variables()
            store_test = store_test.sort_values('영업일자')
            store_test = mv.make_variables_train(data = store_test)
            store_test_sorted = store_test.sort_values('영업일자')

            features = cols + ["매출수량"]
            if len(store_test_sorted) < lookback:
                continue

            recent_df = store_test_sorted[features].iloc[-lookback:].copy()
            if len(recent_df) < lookback:
                continue # lookback 만큼의 데이터가 없으면 예측 안 하고 넘어가기

            ##### 요기서 변수 추가
            last_date = store_test_sorted['영업일자'].iloc[-1]
            recent_df_for_mv = store_test_sorted[features + ['영업장명_메뉴명', '영업일자']].iloc[-lookback:].copy()
            future_df = mv.make_variables_test(date = last_date, test_df = recent_df_for_mv, predict = 7)
            future_df['매출수량'] = 0.0
            full_df = pd.concat([recent_df, future_df[features]], axis = 0)

            # enc_cols 정규화
            for col in enc_cols:
                if col not in full_df.columns:
                    continue
                if col in encoders:
                    le = encoders[col]
                    full_df[col] = le.transform(full_df[col].astype(str)) ##### str로 바꿔봄...
                else:
                    full_df[col] = full_df[col].astype(int)

            # num_cols 스케일링
            full_df[num_cols] = features_scaler.transform(full_df[num_cols])

            x_input_vals = full_df[cols].values
            x_input = x_input_vals[:lookback]
            x_input = torch.tensor([x_input]).float().to(device)

            # 예측 수행
            with torch.no_grad():
                pred_scaled = model(x_input).squeeze().cpu().numpy()

            # 역정규화
            restored = []
            for i in range(predict):
                dummy = np.zeros((1, len(features)))
                dummy[0, features.index("매출수량")] = pred_scaled[i]
                restored_val = target_scaler.inverse_transform(dummy)[0, features.index("매출수량")]
                restored.append(max(restored_val, 0)) # 음수 나오면 0으로 처리

            # 예측일자: TEST_00+1일 ~ TEST_00+7일
            pred_dates = [f"{test_prefix}+{i+1}일" for i in range(predict)]

            for d, val in zip(pred_dates, restored):
                results.append({
                    '영업일자': d,
                    '영업장명_메뉴명': store_menu,
                    '매출수량(lstm)': val
                })

        return pd.DataFrame(results)

#### 예측값 생성

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols, num_cols, device = 'cpu')
    all_preds_lstm.append(pred_lstm)

df_lstm  = pd.concat(all_preds_lstm, ignore_index=True)

         영업일자            영업장명_메뉴명  매출수량
0  2024-06-16  느티나무 셀프BBQ_1인 수저세트     2
1  2024-06-17  느티나무 셀프BBQ_1인 수저세트     0
2  2024-06-18  느티나무 셀프BBQ_1인 수저세트     0
3  2024-06-19  느티나무 셀프BBQ_1인 수저세트     0
4  2024-06-20  느티나무 셀프BBQ_1인 수저세트     4
          영업일자              영업장명_메뉴명  매출수량
28  2024-06-16  느티나무 셀프BBQ_BBQ55(단체)     0
29  2024-06-17  느티나무 셀프BBQ_BBQ55(단체)     0
30  2024-06-18  느티나무 셀프BBQ_BBQ55(단체)     0
31  2024-06-19  느티나무 셀프BBQ_BBQ55(단체)    67
32  2024-06-20  느티나무 셀프BBQ_BBQ55(단체)   124
          영업일자                영업장명_메뉴명  매출수량
56  2024-06-16  느티나무 셀프BBQ_대여료 30,000원     5
57  2024-06-17  느티나무 셀프BBQ_대여료 30,000원     0
58  2024-06-18  느티나무 셀프BBQ_대여료 30,000원     2
59  2024-06-19  느티나무 셀프BBQ_대여료 30,000원     1
60  2024-06-20  느티나무 셀프BBQ_대여료 30,000원     0
          영업일자                영업장명_메뉴명  매출수량
84  2024-06-16  느티나무 셀프BBQ_대여료 60,000원     3
85  2024-06-17  느티나무 셀프BBQ_대여료 60,000원     0
86  2024-06-18  느티나무 셀프BBQ_대여료 60,000원     0
87  2024-06-19  느티나무 셀프BBQ_대여료 60,000원     0
88  202

KeyboardInterrupt: 

In [ ]:
len(df_lstm)

13510

In [ ]:
import re
all_preds_class = []
all_preds_reg = []
all_preds_lstm = []


# 모든 test_*.csv 순회
test_files = sorted(glob.glob('DATA/test/TEST_*.csv'))
predictions = PredictionFunctions()

for path in test_files:
    test_df = pd.read_csv(path)

    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    # 일단 분류 모델 넣고
    pred_class = predictions.predict_class(test_df, models_class, test_prefix, cols, enc_cols)
    all_preds_class.append(pred_class)

    # 1 나오면 회귀 모델 넣고
    pred_reg = predictions.predict_reg(test_df, models_reg, test_prefix, cols, enc_cols)
    all_preds_reg.append(pred_reg)

    # lstm 넣고
    pred_lstm = predictions.predict_lstm(test_df, trained_lstm, test_prefix, cols, enc_cols, num_cols)
    all_preds_lstm.append(pred_lstm)

    print("예측 완료 !")

    # 합치기 (가중치.. 일단은 1.5 / 8.5 정도....)

df_class = pd.concat(all_preds_class, ignore_index = True)
df_reg   = pd.concat(all_preds_reg, ignore_index = True)
df_lstm  = pd.concat(all_preds_lstm, ignore_index = True)

예측 완료 !
예측 완료 !


In [ ]:
full_pred_df = pd.merge(df_class, df_reg, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.rename(columns={'매출수량': '매출수량(reg)'}, inplace=True)

df_lstm_plz = df_lstm.copy()
df_lstm_plz['영업장명_메뉴명'] = df_lstm['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

full_pred_df = pd.merge(df_lstm_plz, full_pred_df, on=['영업일자', '영업장명_메뉴명'], how='outer')
full_pred_df.head()

,영업일자,영업장명_메뉴명,매출수량(lstm),매출여부,매출수량(reg)
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,5.184864,1,10.307570
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),75.649501,0,75.361053
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.046748,1,10.134176
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.486310,1,1.488075
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",0.983028,1,1.748883


In [ ]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.3 + full_pred_df['매출수량(lstm)'] * 0.7,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,6.721676
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),75.649501
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.672977
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.486840
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.212784
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,48.240580
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,24.144339
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,91.218481
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,24.065982


In [ ]:
full_pred_df['매출수량'] = np.where(
    full_pred_df['매출여부'] == 1,
    full_pred_df['매출수량(reg)'] * 0.4 + full_pred_df['매출수량(lstm)'] * 0.6,
    full_pred_df['매출수량(lstm)']
)

full_pred_df.drop(columns=['매출여부', '매출수량(reg)', '매출수량(lstm)'], inplace=True)
full_pred_df

,영업일자,영업장명_메뉴명,매출수량
0,TEST_00+1일,느티나무 셀프BBQ_1인 수저세트,7.233946
1,TEST_00+1일,느티나무 셀프BBQ_BBQ55(단체),75.649501
2,TEST_00+1일,"느티나무 셀프BBQ_대여료 30,000원",8.881719
3,TEST_00+1일,"느티나무 셀프BBQ_대여료 60,000원",1.487016
4,TEST_00+1일,"느티나무 셀프BBQ_대여료 90,000원",1.289370
...,...,...,...
13505,TEST_09+7일,화담숲카페_메밀미숫가루,49.772997
13506,TEST_09+7일,화담숲카페_아메리카노 HOT,27.679980
13507,TEST_09+7일,화담숲카페_아메리카노 ICE,92.439979
13508,TEST_09+7일,화담숲카페_카페라떼 ICE,25.499672


In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
        zip(pred_df['영업일자'], pred_df['영업장명_메뉴명']),
        pred_df['매출수량'].astype(float)
    ))

    final_df = sample_submission.copy()

    menu_cols = final_df.columns[1:]
    final_df[menu_cols] = final_df[menu_cols].astype(float)

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for col in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, col] = pred_dict.get((date, col), 0)

    return final_df

In [ ]:
full_pred_df_notzero = full_pred_df.copy()
full_pred_df_notzero.loc[full_pred_df_notzero['매출수량'].abs() < 1e-9, '매출수량'] = 1

sample_submission = pd.read_csv('DATA/sample_submission.csv')
final_hybrid = convert_to_submission_format(full_pred_df, sample_submission)
final_hybrid.to_csv('baseline_submission_hybrid.csv', index=False, encoding='utf-8-sig')